In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split

**Extract Data from Planes leaving SFO**

In [2]:
# data_2022 = pd.read_csv('archive/Combined_Flights_2022.csv')
# sfo_data_2022 = data_2022[data_2022['Origin'] == 'SFO'].sort_values(by=['FlightDate'])
# sfo_data_2022.to_csv('sfo_data_2022.csv')
# sfo_data_2022

**Feature Selection**

In [3]:
sfo = pd.read_csv('sfo_data_2022.csv')
sfo_columns = ['FlightDate', 'DayOfWeek', 'Operating_Airline', 'Dest', 'DestState', 
               'DepDelay', 'DepDel15', 'DepTimeBlk', 'Distance', 'DistanceGroup']
sfo = sfo.loc[:, sfo_columns]
sfo

,FlightDate,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup
0,2022-01-01,6,OO,MRY,CA,5.0,0.0,2200-2259,77.0,1
1,2022-01-01,6,UA,BNA,TN,75.0,1.0,1600-1659,1969.0,8
2,2022-01-01,6,UA,PHX,AZ,-6.0,0.0,1000-1059,651.0,3
3,2022-01-01,6,UA,PHX,AZ,-5.0,0.0,1600-1659,651.0,3
4,2022-01-01,6,UA,PHX,AZ,-1.0,0.0,1900-1959,651.0,3
...,...,...,...,...,...,...,...,...,...,...
74922,2022-07-31,7,WN,BUR,CA,5.0,0.0,0700-0759,326.0,2
74923,2022-07-31,7,WN,BUR,CA,152.0,1.0,2000-2059,326.0,2
74924,2022-07-31,7,WN,BUR,CA,76.0,1.0,1800-1859,326.0,2
74925,2022-07-31,7,B6,BOS,MA,-8.0,0.0,1000-1059,2704.0,11


**Merging with Weather Data**

In [4]:
sfo['FlightDate'] = sfo['FlightDate'].str.replace('-', '').astype('int')
sfo

,FlightDate,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup
0,20220101,6,OO,MRY,CA,5.0,0.0,2200-2259,77.0,1
1,20220101,6,UA,BNA,TN,75.0,1.0,1600-1659,1969.0,8
2,20220101,6,UA,PHX,AZ,-6.0,0.0,1000-1059,651.0,3
3,20220101,6,UA,PHX,AZ,-5.0,0.0,1600-1659,651.0,3
4,20220101,6,UA,PHX,AZ,-1.0,0.0,1900-1959,651.0,3
...,...,...,...,...,...,...,...,...,...,...
74922,20220731,7,WN,BUR,CA,5.0,0.0,0700-0759,326.0,2
74923,20220731,7,WN,BUR,CA,152.0,1.0,2000-2059,326.0,2
74924,20220731,7,WN,BUR,CA,76.0,1.0,1800-1859,326.0,2
74925,20220731,7,B6,BOS,MA,-8.0,0.0,1000-1059,2704.0,11


In [5]:
weather = pd.read_csv('weather.csv')
weather = weather.dropna(axis=1).drop(['Station', 'Time'], axis=1)
weather = weather.rename({'Date': 'FlightDate', 'Air max': 'Air_Temp_Max', 'min': 'Air_Temp_Min'}, axis=1)
weather

,FlightDate,Precip,Air_Temp_Max,Air_Temp_Min
0,20220101,0.20,56,44
1,20220102,0.21,55,43
2,20220103,0.12,56,43
3,20220104,0.40,55,43
4,20220105,0.07,55,43
...,...,...,...,...
207,20220727,0.00,71,55
208,20220728,0.00,70,56
209,20220729,0.00,71,56
210,20220730,0.00,72,55


In [6]:
sfo_weather = weather.merge(sfo, how='inner', on='FlightDate')
sfo_weather

,FlightDate,Precip,Air_Temp_Max,Air_Temp_Min,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup
0,20220101,0.2,56,44,6,OO,MRY,CA,5.0,0.0,2200-2259,77.0,1
1,20220101,0.2,56,44,6,UA,BNA,TN,75.0,1.0,1600-1659,1969.0,8
2,20220101,0.2,56,44,6,UA,PHX,AZ,-6.0,0.0,1000-1059,651.0,3
3,20220101,0.2,56,44,6,UA,PHX,AZ,-5.0,0.0,1600-1659,651.0,3
4,20220101,0.2,56,44,6,UA,PHX,AZ,-1.0,0.0,1900-1959,651.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74922,20220731,0.0,72,56,7,WN,BUR,CA,5.0,0.0,0700-0759,326.0,2
74923,20220731,0.0,72,56,7,WN,BUR,CA,152.0,1.0,2000-2059,326.0,2
74924,20220731,0.0,72,56,7,WN,BUR,CA,76.0,1.0,1800-1859,326.0,2
74925,20220731,0.0,72,56,7,B6,BOS,MA,-8.0,0.0,1000-1059,2704.0,11


In [7]:
sfo_weather.to_csv('sfo_weather.csv')

**Removing Nan values**

In [14]:
dataset = pd.read_csv('sfo_weather.csv')
dataset = dataset.drop(dataset.columns[0], axis=1)
dataset

,FlightDate,Precip,Air_Temp_Max,Air_Temp_Min,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup
0,20220101,0.2,56,44,6,OO,MRY,CA,5.0,0.0,2200-2259,77.0,1
1,20220101,0.2,56,44,6,UA,BNA,TN,75.0,1.0,1600-1659,1969.0,8
2,20220101,0.2,56,44,6,UA,PHX,AZ,-6.0,0.0,1000-1059,651.0,3
3,20220101,0.2,56,44,6,UA,PHX,AZ,-5.0,0.0,1600-1659,651.0,3
4,20220101,0.2,56,44,6,UA,PHX,AZ,-1.0,0.0,1900-1959,651.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74922,20220731,0.0,72,56,7,WN,BUR,CA,5.0,0.0,0700-0759,326.0,2
74923,20220731,0.0,72,56,7,WN,BUR,CA,152.0,1.0,2000-2059,326.0,2
74924,20220731,0.0,72,56,7,WN,BUR,CA,76.0,1.0,1800-1859,326.0,2
74925,20220731,0.0,72,56,7,B6,BOS,MA,-8.0,0.0,1000-1059,2704.0,11


In [15]:
dataset = dataset[dataset.isnull().any(axis=1) == False]
dataset

,FlightDate,Precip,Air_Temp_Max,Air_Temp_Min,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup
0,20220101,0.2,56,44,6,OO,MRY,CA,5.0,0.0,2200-2259,77.0,1
1,20220101,0.2,56,44,6,UA,BNA,TN,75.0,1.0,1600-1659,1969.0,8
2,20220101,0.2,56,44,6,UA,PHX,AZ,-6.0,0.0,1000-1059,651.0,3
3,20220101,0.2,56,44,6,UA,PHX,AZ,-5.0,0.0,1600-1659,651.0,3
4,20220101,0.2,56,44,6,UA,PHX,AZ,-1.0,0.0,1900-1959,651.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74922,20220731,0.0,72,56,7,WN,BUR,CA,5.0,0.0,0700-0759,326.0,2
74923,20220731,0.0,72,56,7,WN,BUR,CA,152.0,1.0,2000-2059,326.0,2
74924,20220731,0.0,72,56,7,WN,BUR,CA,76.0,1.0,1800-1859,326.0,2
74925,20220731,0.0,72,56,7,B6,BOS,MA,-8.0,0.0,1000-1059,2704.0,11


In [16]:
dataset.columns

Index(['FlightDate', 'Precip', 'Air_Temp_Max', 'Air_Temp_Min', 'DayOfWeek',
       'Operating_Airline', 'Dest', 'DestState', 'DepDelay', 'DepDel15',
       'DepTimeBlk', 'Distance', 'DistanceGroup'],
      dtype='object')

In [17]:
dataset = dataset.drop(columns=['FlightDate'])
dataset

,Precip,Air_Temp_Max,Air_Temp_Min,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup
0,0.2,56,44,6,OO,MRY,CA,5.0,0.0,2200-2259,77.0,1
1,0.2,56,44,6,UA,BNA,TN,75.0,1.0,1600-1659,1969.0,8
2,0.2,56,44,6,UA,PHX,AZ,-6.0,0.0,1000-1059,651.0,3
3,0.2,56,44,6,UA,PHX,AZ,-5.0,0.0,1600-1659,651.0,3
4,0.2,56,44,6,UA,PHX,AZ,-1.0,0.0,1900-1959,651.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
74922,0.0,72,56,7,WN,BUR,CA,5.0,0.0,0700-0759,326.0,2
74923,0.0,72,56,7,WN,BUR,CA,152.0,1.0,2000-2059,326.0,2
74924,0.0,72,56,7,WN,BUR,CA,76.0,1.0,1800-1859,326.0,2
74925,0.0,72,56,7,B6,BOS,MA,-8.0,0.0,1000-1059,2704.0,11


In [19]:
dataset.columns

Index(['Precip', 'Air_Temp_Max', 'Air_Temp_Min', 'DayOfWeek',
       'Operating_Airline', 'Dest', 'DestState', 'DepDelay', 'DepDel15',
       'DepTimeBlk', 'Distance', 'DistanceGroup'],
      dtype='object')

In [20]:
dataset['DepDelayBinary'] = [1 if x else 0 for x in dataset['DepDelay'] > 0]
dataset

,Precip,Air_Temp_Max,Air_Temp_Min,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup,DepDelayBinary
0,0.2,56,44,6,OO,MRY,CA,5.0,0.0,2200-2259,77.0,1,1
1,0.2,56,44,6,UA,BNA,TN,75.0,1.0,1600-1659,1969.0,8,1
2,0.2,56,44,6,UA,PHX,AZ,-6.0,0.0,1000-1059,651.0,3,0
3,0.2,56,44,6,UA,PHX,AZ,-5.0,0.0,1600-1659,651.0,3,0
4,0.2,56,44,6,UA,PHX,AZ,-1.0,0.0,1900-1959,651.0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74922,0.0,72,56,7,WN,BUR,CA,5.0,0.0,0700-0759,326.0,2,1
74923,0.0,72,56,7,WN,BUR,CA,152.0,1.0,2000-2059,326.0,2,1
74924,0.0,72,56,7,WN,BUR,CA,76.0,1.0,1800-1859,326.0,2,1
74925,0.0,72,56,7,B6,BOS,MA,-8.0,0.0,1000-1059,2704.0,11,0


**Splitting Basic Dataset into Train and Test sets**

In [21]:
training_set, testing_set = train_test_split(dataset, test_size=0.30, random_state=88)

In [22]:
training_set

,Precip,Air_Temp_Max,Air_Temp_Min,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup,DepDelayBinary
5246,0.12,57,44,7,AS,SEA,WA,-15.0,0.0,2100-2159,679.0,3,0
55969,0.00,69,53,7,UA,LAS,NV,-5.0,0.0,1900-1959,414.0,2,0
54183,0.00,68,53,2,UA,SNA,CA,-4.0,0.0,1000-1059,372.0,2,0
8995,0.12,57,45,5,DL,SLC,UT,-8.0,0.0,1000-1059,599.0,3,0
36756,0.06,65,50,3,AS,LAS,NV,-5.0,0.0,1700-1759,414.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41708,0.03,67,51,3,UA,PDX,OR,-9.0,0.0,1900-1959,550.0,3,0
63250,0.00,73,55,5,QX,SEA,WA,-9.0,0.0,0600-0659,679.0,3,0
57287,0.01,70,54,3,WN,SAN,CA,95.0,1.0,2000-2059,447.0,2,1
2764,0.10,56,44,7,UA,SNA,CA,15.0,1.0,1900-1959,372.0,2,1


In [23]:
testing_set

,Precip,Air_Temp_Max,Air_Temp_Min,DayOfWeek,Operating_Airline,Dest,DestState,DepDelay,DepDel15,DepTimeBlk,Distance,DistanceGroup,DepDelayBinary
42812,0.03,68,51,6,OO,LAX,CA,-4.0,0.0,1000-1059,337.0,2,0
25091,0.07,64,48,5,WN,LAS,NV,5.0,0.0,1700-1759,414.0,2,1
72694,0.00,72,55,2,AA,DFW,TX,-10.0,0.0,1800-1859,1464.0,6,0
45014,0.01,67,51,5,UA,EUG,OR,-7.0,0.0,2300-2359,451.0,2,0
6653,0.09,56,45,5,OO,SLC,UT,16.0,1.0,0900-0959,599.0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27046,0.11,62,48,3,UA,OGG,HI,-3.0,0.0,1100-1159,2338.0,10,0
12184,0.17,58,46,1,OO,EUG,OR,6.0,0.0,1600-1659,451.0,2,1
47013,0.01,69,52,4,AA,PHX,AZ,3.0,0.0,1200-1259,651.0,3,1
60610,0.00,72,54,5,UA,SAN,CA,144.0,1.0,2200-2259,447.0,2,1


In [24]:
training_set.to_csv('sfo_weather_train.csv',index=False)
testing_set.to_csv('sfo_weather_test.csv',index=False)

**One Hot Encoding Categorical Variables**

In [25]:
encoded_data = pd.get_dummies(dataset, columns = ['DayOfWeek', 'Operating_Airline', 'Dest', 'DestState', 
                                                  'DepTimeBlk', 'DistanceGroup'], drop_first=True)
encoded_data

,Precip,Air_Temp_Max,Air_Temp_Min,DepDelay,DepDel15,Distance,DepDelayBinary,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,...,DepTimeBlk_2300-2359,DistanceGroup_2,DistanceGroup_3,DistanceGroup_4,DistanceGroup_6,DistanceGroup_7,DistanceGroup_8,DistanceGroup_9,DistanceGroup_10,DistanceGroup_11
0,0.2,56,44,5.0,0.0,77.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.2,56,44,75.0,1.0,1969.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0.2,56,44,-6.0,0.0,651.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0.2,56,44,-5.0,0.0,651.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0.2,56,44,-1.0,0.0,651.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74922,0.0,72,56,5.0,0.0,326.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
74923,0.0,72,56,152.0,1.0,326.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
74924,0.0,72,56,76.0,1.0,326.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
74925,0.0,72,56,-8.0,0.0,2704.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


**Splitting Encoded Dataset into Train and Test sets**

In [26]:
training_set_encoded, testing_set_encoded = train_test_split(encoded_data, test_size=0.30, random_state=88)

In [27]:
training_set_encoded

,Precip,Air_Temp_Max,Air_Temp_Min,DepDelay,DepDel15,Distance,DepDelayBinary,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,...,DepTimeBlk_2300-2359,DistanceGroup_2,DistanceGroup_3,DistanceGroup_4,DistanceGroup_6,DistanceGroup_7,DistanceGroup_8,DistanceGroup_9,DistanceGroup_10,DistanceGroup_11
5246,0.12,57,44,-15.0,0.0,679.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
55969,0.00,69,53,-5.0,0.0,414.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
54183,0.00,68,53,-4.0,0.0,372.0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
8995,0.12,57,45,-8.0,0.0,599.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
36756,0.06,65,50,-5.0,0.0,414.0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41708,0.03,67,51,-9.0,0.0,550.0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
63250,0.00,73,55,-9.0,0.0,679.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
57287,0.01,70,54,95.0,1.0,447.0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2764,0.10,56,44,15.0,1.0,372.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [28]:
testing_set_encoded

,Precip,Air_Temp_Max,Air_Temp_Min,DepDelay,DepDel15,Distance,DepDelayBinary,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,...,DepTimeBlk_2300-2359,DistanceGroup_2,DistanceGroup_3,DistanceGroup_4,DistanceGroup_6,DistanceGroup_7,DistanceGroup_8,DistanceGroup_9,DistanceGroup_10,DistanceGroup_11
42812,0.03,68,51,-4.0,0.0,337.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
25091,0.07,64,48,5.0,0.0,414.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
72694,0.00,72,55,-10.0,0.0,1464.0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
45014,0.01,67,51,-7.0,0.0,451.0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
6653,0.09,56,45,16.0,1.0,599.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27046,0.11,62,48,-3.0,0.0,2338.0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
12184,0.17,58,46,6.0,0.0,451.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
47013,0.01,69,52,3.0,0.0,651.0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
60610,0.00,72,54,144.0,1.0,447.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [29]:
training_set_encoded.to_csv('sfo_weather_encoded_train.csv',index=False)
testing_set_encoded.to_csv('sfo_weather_encoded_test.csv',index=False)